In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## DAY-1
data = []
n= int(input('number of employees: '))
for i in range(n):
    print(f"\nEnter details for employee {i+1}:")
    name = input("Name: ")
    age = int(input("Age: "))
    department = input("Department: ")
    salary = int(input("Salary: "))
    
    # append to list as dictionary
    data.append({
        "name": name,
        "age": age,
        "department": department,
        "salary": salary
    })           

df = pd.DataFrame(data)

df.head(3)

df[['name', 'salary']]

avg_sal = df['salary'].mean()
avg_sal

max_sal = df['salary'].max()
print(df['name'][df['salary'] == max_sal])

np.std(df['salary'])

age = np.array(df['age'])
double = age*2
double

In [ ]:
# DAY-3
df = pd.read_csv('/kaggle/input/training2-titanic-data/titanic.csv')
df

In [ ]:
df.isnull().sum()

In [ ]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=['PassengerId', 'Cabin', 'Name', 'Ticket'], inplace=True)

In [ ]:
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum().sum()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode Sex
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df

In [ ]:
df['Embarked'].value_counts()

In [ ]:
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

In [ ]:
df

In [ ]:
df['Embarked_Q'] = df['Embarked_Q'].astype(int)
df['Embarked_S'] = df['Embarked_S'].astype(int)


In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Define models in a dictionary
model = RandomForestClassifier(n_estimators=200,random_state = 42)
# Loop through models
model.fit(X_train, y_train)
y_pred = model.predict(X_test)    
acc = accuracy_score(y_test, y_pred)
print(f" Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))
print("-" * 50)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])
df

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42, stratify=y)

train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class titanicNN(nn.Module):
    def __init__(self, inp):
        super(titanicNN, self).__init__()
        self.fc1 = nn.Linear(inp, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64,128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model = titanicNN(X.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 1000
for EPOCH in range(epochs):
    model.train()
    for x_batch , y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    if (EPOCH+1)%10 == 0:
        print(f"Epoch {EPOCH+1}/{epochs}, Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    y_pred_probs = model(X_test_t)
    y_pred_labels = (y_pred_probs > 0.5).int()
    acc = (y_pred_labels.eq(y_test_t.int()).sum().item()) / y_test_t.shape[0]
    print(f"Test Accuracy: {acc:.4f}")

In [ ]:
#### DAY-4
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = LogisticRegression(
        max_iter=1000, solver='lbfgs', C=1.0, random_state=random_state
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'LogisticRegression',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= LOGISTIC REGRESSION =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Logistic Regression ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()


In [ ]:
### DAY-5
### DECISION TREES

import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = DecisionTreeClassifier(
        criterion="gini",   # or "entropy"
        max_depth=None,     # try limiting to prevent overfitting
        random_state=random_state
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'DecisionTree',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= DECISION TREE =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Decision Tree ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()


In [ ]:
### DAY-5
### RANDOM FOREST

import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model = RandomForestClassifier(
        n_estimators=200,      # number of trees (more trees = better but slower)
        max_depth=10,          # prevent trees from growing too deep
        min_samples_split=5,   # minimum samples to split a node
        min_samples_leaf=2,    # minimum samples per leaf
        max_features="sqrt",   # best practice for classification
        bootstrap=True,        # standard RF bootstrapping
        random_state=random_state,
        n_jobs=-1              # use all CPU cores
    )
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'RandomForest',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_prob),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= RANDOM FOREST =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Random Forest ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()


In [ ]:
### DAY-5
### SUPPORT VECTOR MACHINE (SVC)

import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

def run(show_plot=True, verbose=True, random_state=69):
    data = load_breast_cancer()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state, stratify=y
    )

    # Pipeline: StandardScaler -> SVC
    model = Pipeline([
        ("scaler", StandardScaler()),
        ("svc", SVC(
            kernel="rbf",        # RBF is the default
            C=10.0,              # regularization strength (tune this)
            gamma="scale",       # kernel coefficient
            probability=True,    # enable predict_proba
            random_state=random_state
        ))
    ])

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    metrics = {
        'model': 'SVC',
        'accuracy': accuracy_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'auc': roc_auc_score(y_test, y_prob),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred, digits=4)
    }

    if verbose:
        print('======= SUPPORT VECTOR MACHINE =======')
        print(f"Accuracy: {metrics['accuracy']:.4f}")
        print(f"F1 Score: {metrics['f1']:.4f}")
        print(f"AUC: {metrics['auc']:.4f}")
        print("Confusion Matrix:\n", metrics['confusion_matrix'])
        print("Classification Report:\n", metrics['classification_report'])

    if show_plot:
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        plt.figure()
        plt.plot(fpr, tpr, label=f"AUC = {metrics['auc']:.4f}")
        plt.plot([0, 1], [0, 1], '--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('SVM ROC Curve')
        plt.legend()
        plt.show()

    return metrics


if __name__ == '__main__':
    run()

In [ ]:
### DAY 6
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(8 * 13 * 13, 10)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(-1, 8 * 13 * 13)
        x = self.fc1(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(
    root='/kaggle/working',
    train=True,
    download=True,
    transform=transform
)
testset = torchvision.datasets.MNIST(
    root='/kaggle/working',
    train=False,
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

print("Train batches:", len(trainloader))
print("Test batches:", len(testloader))


In [ ]:
### DAY 8
### REGULARIZED CNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
class regCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)   
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)               
        self.fc1 = nn.Linear(16 * 14 * 14, 10)

    def forward(self, x):
        x = self.pool(self.bn1(torch.relu(self.conv1(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc1(x)
        
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)
testset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = regCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {running_loss/len(trainloader):.4f}")

print("Training done")


In [ ]:
### DAY- 13

# word2vec

import torch 
import torch.nn as nn
import torch.optim as optim

corpus= 'king queen man woman'
words= corpus.split()
vocab= list(set(words))
word_to_idx= {word: i for i, word in enumerate(vocab)}
idx_to_word= {i: word for word,i in word_to_idx.items()}
pairs= [(words[i], words[i+1]) for i in range(len(words)-1)]

class word2vec(nn.Module):
    def __init__(self, vocab_size, embed_dim):
        super().__init__()
        self.emb= nn.Embedding(vocab_size, embed_dim)
        self.linear= nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        return self.linear(self.emb(x))

model = word2vec(len(vocab), 5)
loss_fn= nn.CrossEntropyLoss()
opt= optim.SGD(model.parameters(), lr= 1e-5)
for epoch in range(50):
    total_loss= 0
    for center, context in pairs:
        context_idx= torch.tensor([word_to_idx[context]])
        center_idx= torch.tensor([word_to_idx[center]])

        opt.zero_grad()
        pred= model(center_idx)
        loss= loss_fn(pred, context_idx)
        loss.backward()
        opt.step()
        total_loss+= loss.item()
    print(f'EPOCH {epoch}, loss: {total_loss/len(pairs)}')

embeddings= model.emb.weight.data
print('vector for king:', embeddings[word_to_idx['king']])
print('vector for queen:', embeddings[word_to_idx['queen']])

In [ ]:
###LSTMM

sentences = ['this movie is so ass', 'this movie is so good omg', 
             'i love this movie', 'i hated it']
labels = [0, 1, 1, 0]

# Build vocab
vocab = {w: i+1 for i, w in enumerate(set(' '.join(sentences).split()))}  # +1 so 0 can be used for padding
vocab_size = len(vocab) + 1
maxlen = 6

def encode(s):
    ids= [vocab[w] for w in s.split()]
    return ids[:maxlen] + [0]*D(maxlen- len(ids))

class LSTMClassifier(nn.Module):

In [ ]:
###LSTMMMMM

import torch
import torch.nn as nn
import torch.optim as optim

sentences = ['this movie is so ass', 'this movie is so good omg', 
             'i love this movie', 'i hated it']
labels = [0, 1, 1, 0]

vocab = {w: i+1 for i, w in enumerate(set(' '.join(sentences).split()))}
vocab_size = len(vocab) + 1
maxlen = 6
def encode(s):
    ids = [vocab[w] for w in s.split()]
    return ids[:maxlen] + [0]*max(0, maxlen - len(ids))
X = torch.tensor([encode(s) for s in sentences])
y = torch.tensor(labels)
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, hidden_dim=64, num_classes=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)               
        _, (h_n, _) = self.lstm(x)          
        out = self.fc(h_n.squeeze(0))       
        return out

model = LSTMClassifier(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(20):
    optimizer.zero_grad()
    logits = model(X)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 5 == 0:
        preds = logits.argmax(dim=1)
        acc = (preds == y).float().mean().item()
        print(f"Epoch {epoch+1} | Loss: {loss.item():.4f} | Acc: {acc:.2f}")


In [ ]:
##### DAY 14

import torch
import torch.nn as nn
import torch.nn.functional as F

def attention(q,k,v):
    d= q.size(-1)
    scores = q@k.transpose(-2,-1)/d ** 0.5
    attn = F.softmax(scores, dim=-1)
    return attn @ v, attn

q= torch.randn(1,4,8)
out, attn= attention(q,q,q)
print(out.shape, attn.shape)

In [ ]:
### TRANSFORMER

from transformers import pipeline

summarizer = pipeline('summarization', model= 'facebook/bart-large-cnn')
text= 'Climate change is accelerating the frequency and intensity of extreme weather events around the world. Rising global temperatures have contributed to longer heatwaves, stronger hurricanes, and more devastating wildfires. According to the Intergovernmental Panel on Climate Change (IPCC), human activities, especially the burning of fossil fuels, are the primary drivers of these changes. Governments are being urged to adopt cleaner energy sources, reduce greenhouse gas emissions, and invest in climate-resilient infrastructure. Without immediate and coordinated action, scientists warn that the impacts on ecosystems, economies, and human health will become increasingly severe and irreversible.'
output= summarizer(text, max_length= 40, min_length= 10)[0]['summary_text']
print(output)

In [ ]:
###DAY 15
from transformers import MarianMTModel, MarianTokenizer

# Load model and tokenizer
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')
tok = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')

# English sentences to translate
sentences = [
    'good morning',
    'how are you',
    'i hate you',
    'fuck you, you asshole'
]

# Tokenize the input batch
batch = tok(sentences, return_tensors='pt', padding=True)

# Generate translations
translated = model.generate(**batch, max_length=64)

# Decode and print translations
for s, t in zip(sentences, translated):
    print(f'{s} --> {tok.decode(t, skip_special_tokens=True)}')


In [ ]:
# DAY 15

# Morse code mapping dictionary
morse_map = {
    'a': '.-', 'b': '-...', 'c': '-.-.', 'd': '-..', 'e': '.', 
    'f': '..-.', 'g': '--.', 'h': '....', 'i': '..', 'j': '.---',
    'k': '-.-', 'l': '.-..', 'm': '--', 'n': '-.', 'o': '---',
    'p': '.--.', 'q': '--.-', 'r': '.-.', 's': '...', 't': '-',
    'u': '..-', 'v': '...-', 'w': '.--', 'x': '-..-', 'y': '-.--',
    'z': '--..', '0': '-----', '1': '.----', '2': '..---', '3': '...--',
    '4': '....-', '5': '.....', '6': '-....', '7': '--...', '8': '---..',
    '9': '----.', ' ': '/', ',': '--..--', '.': '.-.-.-', '?': '..--..',
    '!': '-.-.--'
}

def text_to_morse(text):
    return ' '.join(morse_map.get(c.lower(), '') for c in text)

# English sentences to "translate"
sentences = [
    'good morning',
    'how are you',
    'i hate you',
    'fuck you, you asshole'
]

# Translate and print
for s in sentences:
    print(f"{s} --> {text_to_morse(s)}")


In [ ]:
# DAY 15

from transformers import MarianMTModel, MarianTokenizer
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-hi')
tok = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

sentences = [
    'good morning',
    'how are you',
    'i hate you',
    'fuck you, you asshole'
]
batch = tok(sentences, return_tensors='pt', padding=True)
translated = model.generate(**batch, max_length=64)
for s, t in zip(sentences, translated):
    print(f'{s} --> {tok.decode(t, skip_special_tokens=True)}')


In [ ]:
# English to German
from transformers import MarianMTModel, MarianTokenizer

print("==== Normal Translation ====")
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')
tok = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')

sentences = [
    'Good Morning!',
    'How are you?',
    'Is your dog okay now?'
]

batch = tok(sentences, return_tensors='pt', padding=True)
translated = model.generate(**batch)

print("Translating English to German!!")
for s, t in zip(sentences, translated):
    print(f"{s} -> {tok.decode(t, skip_special_tokens=True)}")


# English to Hindi
print("==== Idioms Translation ====")
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-hi')
tok = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

sentences = [
    'Good Morning!',
    'How are you?',
    'Is your dog okay now?',
    'This is not your cup of tea',
    'Now ball is in your court'
]

batch = tok(sentences, return_tensors='pt', padding=True)
translated = model.generate(**batch)

print("Translating English to Hindi!!")
for s, t in zip(sentences, translated):
    print(f"{s} -> {tok.decode(t, skip_special_tokens=True)}")


# Reverse Translation
print("==== Reverse Translation ====")
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-de')
tok = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-de')

sentences = [
    'Good Morning!',
    'How are you?',
    'Is your dog okay now?'
]

batch = tok(sentences, return_tensors='pt', padding=True)
translated = model.generate(**batch)
print("Translating English to German!!")
de  = []
for s, t in zip(sentences, translated):
    print(f"{s} -> {tok.decode(t, skip_special_tokens=True)}")
    de.append(tok.decode(t, skip_special_tokens=True))


model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-de-en')
tok = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-de-en')

sentences = de

batch = tok(sentences, return_tensors='pt', padding=True)
translated = model.generate(**batch)

print("Translating German to English!!")
for s, t in zip(sentences, translated):
    print(f"{s} -> {tok.decode(t, skip_special_tokens=True)}")

In [17]:
from transformers import MarianMTModel, MarianTokenizer

model_name = 'Helsinki-NLP/opus-mt-en-hi' 
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
sentences = ["i love my dog"]
batch = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)
translated = model.generate(**batch)
decoded = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
final_translation = " ".join(decoded)
final_translation


'मैं अपने कुत्ते से प्यार'

In [1]:
from datasets import load_dataset
dataset= load_dataset('Abirate/english_quotes')
print(dataset['train'])

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 2508
})


In [2]:
from transformers import AutoModelForCausalLM , AutoTokenizer , TrainingArguments , Trainer
from peft import LoraConfig , get_peft_model
from datasets import load_dataset

dataset = load_dataset("Abirate/english_quotes")

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

lora_config = LoraConfig(task_type = 'CAUSAL_LM' , r = 8 , lora_alpha = 16 , lora_dropout = 0.1)

model = get_peft_model(model, lora_config)

def tokenize(batch):
    return tokenizer(batch['quotes'] , truncation = True , padding = 'max_length' , max_length = 64)

tokenized = dataset.map(tokenize , batched = True)

args = TrainingArguments(
    "gpt2-tune" ,
    per_device_train_batch_size = 4 ,
    num_train_epochs = 1 ,
    logging_steps = 10
)

trainer = Trainer(
    model = model ,
    args = args ,
    train_dataset = tokenized['train'].select(range(200))
    )

trainer.train()

2025-09-24 04:55:29.095371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758689729.310487      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758689729.372438      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

KeyError: 'quotes'